In [15]:
import nfl_data_py as nfl
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
pd.set_option("display.max_columns", None)
import sklearn
from analysis import PositionReport
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [24]:
# List of functions used in the notebook

def execute_statement(sql: str):
    with psycopg2.connect(host="localhost", database="thefantasybot", user="tbakely") as conn:
         df = pd.read_sql(sql, conn)
         return df

def get_corr_heatmap(df: pd.DataFrame, ax = None, show_plot = True):
    exclude_words = ["season", "week"]
    numerical_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64']\
                      and not any(word in col.lower() for word in exclude_words)]
    correlation_matrix = df[numerical_cols].corr()
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    if show_plot:
        sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5, mask=mask, ax=ax)
        plt.show()
    else:
        return sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5, mask=mask, ax=ax)

In [25]:
weekly_sql = """
select
	wd.player_id,
	wd.player_name,
	position,
	recent_team,
	wd.season,
	wd.week,
	carries,
	rushing_yards,
	rushing_tds,
	rushing_fumbles,
	rushing_fumbles_lost,
	rushing_first_downs,
	rushing_epa,
	efficiency,
	percent_attempts_gte_eight_defenders,
	avg_time_to_los,
	rush_yards_over_expected,
	avg_rush_yards,
	rush_yards_over_expected_per_att,
	rush_pct_over_expected,
	wd.receptions,
	wd.targets,
	receiving_yards,
	receiving_tds,
	receiving_fumbles,
	receiving_fumbles_lost,
	receiving_air_yards,
	receiving_yards_after_catch,
	receiving_first_downs,
	receiving_epa,
	racr,
	target_share,
	air_yards_share,
	wopr,
	offense_snaps,
	offense_pct,
    redzone.redzone,
	(carries + wd.targets) as total_usage,
    wd.fantasy_points,
    wd.fantasy_points_ppr,
	roof,
	surface,
	weather_hazards,
	temp,
	humidity,
	wind_speed
from archive_data.weekly_data wd
left join archive_data.offense_snap_counts os
on wd.player_id = os.id
and wd.season = os.season
and wd.week = os.week
left join archive_data.ngs_rushing_data ngsr
on wd.player_id = ngsr.player_gsis_id
and wd.season = ngsr.season
and wd.week = ngsr.week
left join archive_data.ngs_receiving_data ngsp
on wd.player_id = ngsp.player_gsis_id
and wd.season = ngsp.season
and wd.week = ngsp.week
left join (select distinct rusher_player_id, game_id, season, week from archive_data.full_pbp) game_id
on wd.player_id = game_id.rusher_player_id
and wd.season = game_id.season
and wd.week = game_id.week
left join archive_data.game_data
on game_data.game_id = game_id.game_id
left join archive_data.redzone_snaps redzone
on wd.player_id = redzone.player_id
and wd.season = redzone.season
and wd.week = redzone.week
where position in ('WR', 'RB', 'TE')
and wd.season between 2016 and 2022;
"""

weekly_wr = """
select 
wr.player_name, 
wr.position, 
wr.season,
wr.week,
offense_snaps,
offense_pct,
target_share,
targets,
receiving_epa,
redzone
from current_season_data.weekly_wr wr
left join current_season_data.redzone_snaps rz
on wr.player_id = rz.player_id
and wr.season = rz.season
and wr.week = rz.week
"""

weekly_rb = """
select
rb.player_name,
rb.position,
rb.season,
rb.week,
offense_snaps,
offense_pct,
total_usage,
rushing_epa,
rush_yards_over_expected_per_att,
rush_pct_over_expected,
target_share,
receiving_epa,
redzone
from current_season_data.weekly_rb rb
left join current_season_data.redzone_snaps rz
on rb.player_id = rz.player_id
and rb.season = rz.season
and rb.week = rz.week
"""

weekly_te = """
select 
te.player_name, 
te.position, 
te.season,
te.week,
offense_snaps,
offense_pct,
target_share,
targets,
receiving_epa,
redzone
from current_season_data.weekly_te te
left join current_season_data.redzone_snaps rz
on te.player_id = rz.player_id
and te.season = rz.season
and te.week = rz.week
"""


### Correlation heatmap for weekly position data

In [66]:
wr_df = execute_statement(weekly_wr)
rb_df = execute_statement(weekly_rb)
te_df = execute_statement(weekly_te)

fig, axs = plt.subplots(3, figsize=(10,20))
get_corr_heatmap(wr_df, ax=axs[0], show_plot=False)
axs[0].set_title("WR")
get_corr_heatmap(rb_df, ax=axs[1], show_plot=False)
axs[1].set_title("RB")
get_corr_heatmap(te_df, ax=axs[2], show_plot=False)
axs[2].set_title("TE")
plt.tight_layout()
plt.savefig("positional_heatmaps.png", bbox_inches='tight', dpi=300)
plt.close()




In [161]:
weekly = execute_statement(weekly_wr)
weekly_rb = execute_statement(weekly_rb)

In [8]:
from analysis import PositionReport as PR

In [9]:
rb_report = PR("RB")
wr_report = PR("WR")
te_report = PR("TE")

In [11]:
rb_report.get_report_by_week(18).head(50)

,player_name,position,season,week,score
495,N.Harris,RB,2023,18,79.594993
217,A.Jones,RB,2023,18,69.429154
420,J.Conner,RB,2023,18,62.699692
1208,D.Henry,RB,2023,18,45.090471
43,J.Mixon,RB,2023,18,43.300189
167,J.Taylor,RB,2023,18,38.709368
634,B.Hall,RB,2023,18,38.210341
130,T.Pollard,RB,2023,18,36.113026
1039,E.Mitchell,RB,2023,18,34.430263
203,S.Barkley,RB,2023,18,28.852016


In [17]:
rb_report.compare_reports(9, 18).head(50)

,player_name,position,season,score
18,C.McCaffrey,RB,2023,57.152531
77,K.Williams,RB,2023,57.037436
45,I.Pacheco,RB,2023,45.356065
5,A.Jones,RB,2023,35.140797
47,J.Conner,RB,2023,35.008488
97,R.White,RB,2023,32.165288
86,N.Harris,RB,2023,30.602410
59,J.Mixon,RB,2023,30.106976
99,S.Barkley,RB,2023,29.678462
108,T.Pollard,RB,2023,28.941371


In [13]:
wr_report.get_report_by_week(18).head(36)

,player_name,position,season,week,score
1504,C.Lamb,WR,2023,18,113.327815
315,D.Adams,WR,2023,18,75.310165
933,T.Hill,WR,2023,18,74.364426
481,J.Jefferson,WR,2023,18,71.719919
1381,B.Cooks,WR,2023,18,71.078003
758,C.Ridley,WR,2023,18,64.760350
1238,A.Iosivas,WR,2023,18,64.237855
636,N.Collins,WR,2023,18,63.256805
420,D.Slayton,WR,2023,18,58.369248
559,A.St. Brown,WR,2023,18,58.156052


In [21]:
wr_report.compare_reports(14,18).head(36)

,player_name,position,season,score
26,C.Kupp,WR,2023,85.214413
27,C.Lamb,WR,2023,79.158048
35,D.Adams,WR,2023,75.270410
8,A.St. Brown,WR,2023,73.097073
1,A.Cooper,WR,2023,70.991402
146,R.Rice,WR,2023,65.583916
30,C.Ridley,WR,2023,64.318506
133,P.Nacua,WR,2023,61.822970
53,D.Samuel,WR,2023,61.806278
159,T.Hill,WR,2023,61.481619


In [18]:
import pandas as pd


te_report.get_report_by_week(18).head(24)

,player_name,position,season,week,score
882,T.McBride,TE,2023,18,105.533641
386,A.Hooper,TE,2023,18,93.567119
965,C.Kmet,TE,2023,18,89.074555
605,E.Engram,TE,2023,18,81.585021
824,J.Ferguson,TE,2023,18,73.845364
900,D.Knox,TE,2023,18,53.196937
469,D.Schultz,TE,2023,18,52.966632
995,J.Smith,TE,2023,18,47.167098
731,S.LaPorta,TE,2023,18,46.951319
139,W.Dissly,TE,2023,18,45.880619


In [20]:
te_report.compare_reports(9, 18).head(24)

,player_name,position,season,score
39,G.Kittle,TE,2023,68.771086
90,S.LaPorta,TE,2023,66.322120
33,E.Engram,TE,2023,66.037437
12,C.Kmet,TE,2023,65.008417
100,T.McBride,TE,2023,64.335515
50,J.Ferguson,TE,2023,63.119112
26,D.Njoku,TE,2023,61.878482
98,T.Kelce,TE,2023,61.520023
96,T.Hockenson,TE,2023,60.947573
16,C.Otton,TE,2023,58.878900


In [286]:
def get_corr_heatmap(df: pd.DataFrame):
    exclude_words = ["season", "week"]
    numerical_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64']\
                      and not any(word in col.lower() for word in exclude_words)]
    correlation_matrix = df[numerical_cols].corr()
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5, mask=mask)
    plt.show()

In [4]:
rushing_cols = [
    "rushing_yards",
    "rushing_tds",
    "rushing_fumbles",
    "rushing_fumbles_lost",
    "rushing_first_downs",
    "rushing_epa",
    "efficiency",
    "percent_attempts_gte_eight_defenders",
    "avg_time_to_los",
    "rush_yards_over_expected",
    "avg_rush_yards",
    "rush_yards_over_expected_per_att",
    "rush_pct_over_expected",
    "targets",
    "offense_snaps",
    "offense_pct",
    "redzone",
    "total_usage",
]

weekly[weekly["player_name"].isna()]

,player_id,player_name,position,recent_team,season,week,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,racr,target_share,air_yards_share,wopr,offense_snaps,offense_pct,redzone,total_usage,fantasy_points,fantasy_points_ppr,roof,surface,weather_hazards,temp,humidity,wind_speed
0,00-0020337,None,WR,BAL,2016,1,0,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,8,19.0,0,0.0,0.0,41.0,12.0,0.0,-1.190828,0.463415,0.250000,0.166667,0.491667,45.0,0.66,1.0,8,1.9,6.9,None,None,None,NaN,NaN,NaN
1,00-0020337,None,WR,BAL,2016,2,0,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,64.0,0,0.0,0.0,80.0,5.0,3.0,0.058890,0.800000,0.133333,0.164948,0.315464,53.0,0.71,NaN,6,6.4,9.4,None,None,None,NaN,NaN,NaN
2,00-0020337,None,WR,BAL,2016,3,0,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,11,87.0,0,0.0,0.0,66.0,39.0,5.0,3.585929,1.318182,0.275000,0.246269,0.584888,40.0,0.60,NaN,11,8.7,16.7,None,None,None,NaN,NaN,NaN
3,00-0020337,None,WR,BAL,2016,4,0,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,11,111.0,1,0.0,0.0,88.0,60.0,4.0,4.417709,1.261364,0.220000,0.270769,0.519538,73.0,0.82,2.0,11,17.1,25.1,None,None,None,NaN,NaN,NaN
4,00-0020337,None,WR,BAL,2016,5,0,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,29.0,0,0.0,0.0,27.0,2.0,2.0,0.981787,1.074074,0.063830,0.079882,0.151662,13.0,0.18,NaN,3,2.9,5.9,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15040,00-0032482,None,WR,CHI,2016,17,1,4.0,0,0.0,0.0,0.0,-0.529760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,11.0,0,0.0,0.0,3.0,8.0,1.0,0.881909,3.666667,0.050000,0.017442,0.087209,2.0,0.03,NaN,2,1.5,2.5,dome,sportturf,NONE,NaN,NaN,NaN
15251,00-0032636,None,RB,GB,2016,7,2,6.0,0,0.0,0.0,0.0,0.127916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0.0,0,0.0,0.0,10.0,0.0,0.0,-0.739813,0.000000,0.017857,0.032154,0.049294,NaN,NaN,NaN,3,0.6,0.6,outdoors,grass,NONE,47.0,0.63,10.0
15252,00-0032636,None,RB,GB,2016,8,4,10.0,0,0.0,0.0,1.0,-0.476560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4,1.0,1.0,dome,fieldturf,NONE,NaN,NaN,NaN
15253,00-0032636,None,RB,GB,2016,9,4,16.0,0,0.0,0.0,0.0,-0.326300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1.6,1.6,outdoors,grass,NONE,68.0,0.55,7.0
